## 해상도 10, -40 ~ 40

### package 불러오기 (환경 설정)

In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
import numpy as np

### 변수 설정

In [2]:
# 변수 설정
BATCH_SIZE = 40
BATCH_MOMENTM = 0.9
EPOCH = 5
MIN_ANGLE, MAX_ANGLE, RESOLUTION = -40, 40, 10
NUM_CLASS = (MAX_ANGLE-MIN_ANGLE) // RESOLUTION + 1

print(NUM_CLASS)

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

9


### 데이터 셋 및 모델 설정

#### 데이터 셋 설정

In [3]:
class RadarDataset(Dataset):
  def __init__(self, transform, data, label):
    super(RadarDataset, self).__init__()
    self.transform = transform
    self.label = self.transform(label)
    self.data = self.transform(data).type(torch.float)

  def __len__(self, ):
    return self.data.shape[0]

  def __getitem__(self, index):   
    return self.data[index], self.label[index]


#### 모델 설정

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
    
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=BATCH_MOMENTM),
            nn.ReLU())
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=BATCH_MOMENTM),
            nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=BATCH_MOMENTM),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=BATCH_MOMENTM),
            nn.ReLU())
        
        self.fc1 = nn.Sequential(
            nn.Linear(4*4*256, 4096),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
                
        self.fc3 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        
        self.fc4 = nn.Sequential(
            nn.Linear(1024, NUM_CLASS),
            nn.ReLU()
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        
        return out

### 학습된 모델 불러오기

In [9]:
model = ConvNet().to(device)
model.load_state_dict(torch.load(f'./model_{RESOLUTION}_{MAX_ANGLE}.ckpt'))

<All keys matched successfully>

### 실험 데이터 불러오기

In [10]:
Rx_sig = np.load('../../data/0/output_COV_0.npy')
angle = np.load('../../data/0/output_angle_0.npy')

x_test = np.array([[(np.real(Rxx)), (np.imag(Rxx)), ((np.angle(Rxx)))] for Rxx in Rx_sig])
x_test_torch= torch.from_numpy(x_test).type(torch.float).to(device)
y_test = angle.astype(int)

### 실험 데이터 검증

In [11]:
def cal_mapping_val(pred, test):
    pred_list, test_list = list(pred), list(test)
    mapping_list, mapping_val = [], []
    
    for val_test in test_list:
        min_val = float('inf')
        
        for val_pred in pred_list:
            result = abs(val_test - val_pred)
            
            if result < min_val:
                min_val = result
                mapping_val = val_pred
                
        mapping_list.append(abs(val_test - mapping_val))
        pred_list.remove(mapping_val)
    
    return mapping_list

In [12]:
angle_range = torch.arange(MIN_ANGLE, MAX_ANGLE+1, RESOLUTION)

correct = 0
TOLERANCE_ANGLE = 0

model.eval() 
output = model(x_test_torch)
results = map(lambda matrix: torch.topk(matrix.reshape(-1), k=1, largest=True), output) 
predicted = torch.tensor([list(map(lambda x: x.item(), indices)) for _, indices in results])

pred_value = angle_range[predicted].T.numpy().reshape(-1)
real_value = y_test.T.reshape(-1)
print("percent".center(40, "-"))
result = np.array(list((map(cal_mapping_val, angle_range[predicted], y_test))))
correct += (result <= RESOLUTION * TOLERANCE_ANGLE).sum()
print(correct/len(real_value) * 100, end='\n\n')
# print(f'{(pred_value==real_value).sum()/len(real_value) * 100}%', end='\n\n')

print("data".center(40, "-"))
print("predict")
print(pred_value);print()
print("Real")
print(real_value)

----------------percent-----------------
5.1

------------------data------------------
predict
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  